In [44]:
import pandas as pd
CHINA_DATASET = "heart_attack_china.csv"
US_DATASET = "heart_attack_us.csv"
INDIA_DATASET = "heart_attack_india.csv"
INDONESIA_DATASET = "heart_attack_indonesia.csv"
china = pd.read_csv(CHINA_DATASET, sep=",")
unitedStates = pd.read_csv(US_DATASET, sep=",")
india = pd.read_csv(INDIA_DATASET, sep=",")
indonesia = pd.read_csv(INDONESIA_DATASET, sep=",")

In [45]:
china['State'] = "China"
india['State'] = "India"
unitedStates['State'] = "United States"
indonesia['State'] = "Indonesia"

unitedStates["PatientID"] = range(1, len(unitedStates)+1)
indonesia["PatientID"] = range(1, len(indonesia)+1)

In [46]:
(
    unitedStates.drop(
        columns = ['EducationLevel',
                   'ST_Depression',
                   'EmploymentStatus',
                   'MaritalStatus', 
                   'Ethnicity',
                   'Medication',
                   'ChestPainType',
                   'ECGResults',
                   'ExerciseInducedAngina',
                   'Slope',
                   'NumberOfMajorVessels',
                   'Thalassemia',
                   'StrokeHistory',
                   'Residence',
                   'MaxHeartRate',
                   'HeartRate'
                  ],
        inplace=True
    )
)

In [47]:
unitedStates.rename(columns = {'BMI':'Obesity', 'Income':'IncomeLevel', 'Patient_ID':'PatientID', 'Diet':'HealthyDiet'}, inplace=True)

In [48]:
(
    china.drop(
        columns = ['Education_Level',
                   'Employment_Status',
                   'Air_Pollution_Exposure',
                   'Healthcare_Access',
                   'Rural_or_Urban',
                   'Region',
                   'Province',
                   'Hospital_Availability',
                   'TCM_Use',
                   'Chronic_Kidney_Disease',
                   'CVD_Risk_Score'
                  ],
        inplace=True
    )
)

In [49]:
china.rename(
    columns = { 
        'Patient_ID':'PatientID',
        'Smoking_Status':'Smoker',
        'Cholesterol_Level':'Cholesterol',
        'Physical_Activity': 'PhysicalActivity',
        'Diet_Score':'HealthyDiet',
        'Income_Level':'IncomeLevel',
        'Blood_Pressure':'BloodPressure',
        'Previous_Heart_Attack': 'PreviousHeartAttack',
        'Family_History_CVD': 'FamilyHistory',
        'Stress_Level':'StressLevel',
        'Alcohol_Consumption': 'AlcoholConsumption',
        'Heart_Attack' : 'Outcome'
    },
    inplace=True
)

In [50]:
(
    india.drop(
        columns = ['State_Name',
                   'Triglyceride_Level',
                   'LDL_Level',
                   'HDL_Level',
                   'Diastolic_BP',
                   'Air_Pollution_Exposure',
                   'Healthcare_Access',
                   'Emergency_Response_Time',
                   'Health_Insurance'
                  ],
        inplace=True
    )
)

In [51]:
india.rename(
    columns = {
        'Patient_ID':'PatientID',
        'Smoking':'Smoker',
        'Alcohol_Consumption': 'AlcoholConsumption',
        'Physical_Activity': 'PhysicalActivity',
        'Diet_Score':'HealthyDiet',
        'Cholesterol_Level':'Cholesterol',
        'Systolic_BP':'BloodPressure',
        'Family_History':'FamilyHistory',
        'Stress_Level':'StressLevel',
        'Heart_Attack_History':'PreviousHeartAttack',
        'Annual_Income':'IncomeLevel',
        'Heart_Attack_Risk':'Outcome'
    },
    inplace=True
)

In [52]:
(
    indonesia.drop(
        columns = ['region',
                   'waist_circumference',
                   'air_pollution_exposure',
                   'sleep_hours',
                   'blood_pressure_diastolic',
                   'fasting_blood_sugar',
                   'cholesterol_hdl',
                   'cholesterol_ldl',
                   'triglycerides',
                   'EKG_results',
                   'medication_usage',
                   'participated_in_free_screening'
                  ],
        inplace=True
    )
)

In [53]:
indonesia.rename(
    columns = {
        'age':'Age',
        'gender':'Gender',
        'income_level': 'IncomeLevel',
        'hypertension':'Hypertension',
        'diabetes':'Diabetes',
        'cholesterol_level':'Cholesterol',
        'obesity':'Obesity',
        'family_history':'FamilyHistory',
        'smoking_status':'Smoker',
        'alcohol_consumption':'AlcoholConsumption',
        'physical_activity':'PhysicalActivity',
        'dietary_habits':'HealthyDiet',
        'stress_level':'StressLevel',
        'blood_pressure_systolic':'BloodPressure',
        'previous_heart_disease':'PreviousHeartAttack',
        'heart_attack':'Outcome',
    },
    inplace=True
)

In [54]:
dfs = {
    "China": china,
    "India": india,
    "United States": unitedStates,
    "Indonesia" : indonesia
}

# insieme di tutte le colonne
all_columns = set().union(*(df.columns for df in dfs.values()))

# colonne mancanti per ogni dataset
diff_columns = {
    name: sorted(all_columns - set(df.columns))
    for name, df in dfs.items()
}

diff_columns

{'China': [], 'India': [], 'United States': [], 'Indonesia': []}

In [55]:
initial_memory = china.memory_usage(deep=True).sum()

china['Age'] = china['Age'].astype('int16')
china['BloodPressure'] = china['BloodPressure'].astype('int16')
china['Gender'] = china['Gender'].astype('category')
china['Smoker'] = china['Smoker'].map({'Smoker':True, 'Non-Smoker':False}).astype('bool')
china["Hypertension"] = china["Hypertension"].map({"Yes": True, "No": False}).astype('bool')
china["Diabetes"] = china["Diabetes"].map({"Yes": True, "No": False}).astype('bool')
china["Obesity"] = china["Obesity"].map({"Yes": True, "No": False}).astype('bool')
china['Cholesterol'] = china['Cholesterol'].astype('category')
china['PhysicalActivity'] = china['PhysicalActivity'].map({"High": True, "Medium": True, "Low": False})
china['HealthyDiet'] = china['HealthyDiet'].map({"Healthy": True, "Moderate": True, "Poor": False})
china['StressLevel'] = china['StressLevel'].astype('category')
china['AlcoholConsumption'] = china['AlcoholConsumption'].map({'Yes':True, 'No':False}).astype('bool')
china['FamilyHistory'] = china['FamilyHistory'].map({'Yes':True, 'No':False}).astype('bool')
china['IncomeLevel'] = china['IncomeLevel'].astype('category')
china['PreviousHeartAttack'] = china['PreviousHeartAttack'].map({'Yes':True, 'No':False}).astype('bool')
china['Outcome'] = china['Outcome'].map({'Yes':True, 'No':False}).astype('bool')
china['State'] = china['State'].astype('category')

end_memory = china.memory_usage(deep=True).sum()
china

,PatientID,Age,Gender,Smoker,Hypertension,Diabetes,Obesity,Cholesterol,PhysicalActivity,HealthyDiet,StressLevel,AlcoholConsumption,FamilyHistory,IncomeLevel,BloodPressure,PreviousHeartAttack,Outcome,State
0,1,55,Male,False,False,False,True,Normal,True,True,Low,True,False,Low,104,False,False,China
1,2,66,Female,True,True,False,False,Low,True,True,Medium,False,True,Middle,142,False,False,China
2,3,69,Female,True,False,False,False,Low,True,True,Low,False,False,High,176,False,False,China
3,4,45,Female,True,False,True,False,Normal,False,True,Medium,True,False,Low,178,True,False,China
4,5,39,Female,True,False,False,False,Normal,True,True,Low,False,False,Middle,146,False,False,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239261,239262,53,Male,False,False,False,False,Normal,True,True,Medium,False,False,Low,167,False,True,China
239262,239263,48,Female,False,False,False,True,Normal,True,False,High,False,False,Low,179,False,False,China
239263,239264,57,Male,True,True,False,False,Normal,True,False,Low,False,False,Middle,136,False,False,China
239264,239265,42,Female,False,False,False,False,High,True,False,High,False,True,Middle,159,False,True,China


In [56]:
china_memory_optimization_coeff = initial_memory / end_memory
china_memory_optimization_coeff

np.float64(30.22052834204339)

In [57]:
unitedStates['Gender'] = unitedStates['Gender'].astype('category')

bins = [-float('inf'), 200, 240, float('inf')]
labels = ['Low', 'Normal', 'High']
unitedStates['Cholesterol'] = pd.cut(
    unitedStates['Cholesterol'],
    bins=bins,
    labels=labels,
    right=False
).astype('category')

unitedStates['Obesity'] = unitedStates['Obesity'] >= 30
unitedStates['Smoker'] = unitedStates['Smoker'].map({1:True, 0:False}).astype('bool')
unitedStates['Diabetes'] = unitedStates['Diabetes'].map({1:True, 0:False}).astype('bool')
unitedStates['Hypertension'] = unitedStates['Hypertension'].map({1:True, 0:False}).astype('bool')
unitedStates['FamilyHistory'] = unitedStates['FamilyHistory'].map({1:True, 0:False}).astype('bool')
unitedStates['PhysicalActivity'] = unitedStates['PhysicalActivity'] > 2
unitedStates['AlcoholConsumption'] = unitedStates['AlcoholConsumption'] > 2
china['HealthyDiet'] = china['HealthyDiet'].map({"Healthy": True, "Moderate": True, "Unhealthy": False})

bins = [0, 2, 4, 7]
labels = ['Low', 'Medium', 'High']
unitedStates['StressLevel'] = pd.cut(
    unitedStates['StressLevel'],
    bins=bins,
    labels=labels,
    right=False
).astype('category')

bins = [0, 40000, 100000, float("inf")]
labels = ["Low", "Middle", "High"]

unitedStates["IncomeLevel"] = pd.cut(
    unitedStates["IncomeLevel"],
    bins=bins,
    labels=labels,
    right=False
)

unitedStates['PreviousHeartAttack'] = unitedStates['PreviousHeartAttack'].map({1:True, 0:False}).astype('bool')
unitedStates['Outcome'] = unitedStates['Outcome'].map({'Heart Attack':True, 'No Heart Attack':False}).astype('bool')

,Age,Gender,Cholesterol,BloodPressure,Obesity,Smoker,Diabetes,Hypertension,FamilyHistory,PhysicalActivity,AlcoholConsumption,HealthyDiet,StressLevel,IncomeLevel,PreviousHeartAttack,Outcome,State,PatientID
0,31,Male,Low,162,False,False,True,False,False,True,False,Unhealthy,Low,Middle,False,False,United States,1
1,69,Male,Normal,148,True,True,True,False,False,False,False,Unhealthy,High,Middle,False,False,United States,2
2,34,Female,Low,161,True,False,False,True,True,False,True,Healthy,Medium,High,True,True,United States,3
3,53,Male,High,134,True,False,True,True,False,True,False,Healthy,Medium,Middle,True,False,United States,4
4,57,Female,Normal,140,True,False,True,False,False,True,False,Moderate,Low,Middle,True,True,United States,5
5,41,Male,Low,154,True,False,True,False,True,True,False,Moderate,High,High,False,True,United States,6
6,45,Female,Normal,168,False,False,False,False,False,False,False,Healthy,High,Middle,True,False,United States,7
7,61,Female,Normal,113,False,False,True,False,True,True,True,Healthy,High,Middle,True,False,United States,8
8,45,Male,Normal,114,False,False,True,False,True,False,True,Unhealthy,Medium,High,False,True,United States,9
9,30,Male,Low,130,False,True,True,False,False,True,False,Moderate,Medium,High,False,False,United States,10


In [19]:
unitedStates

,Age,Gender,Cholesterol,BloodPressure,Obesity,Smoker,Diabetes,Hypertension,FamilyHistory,PhysicalActivity,AlcoholConsumption,HealthyDiet,StressLevel,IncomeLevel,PreviousHeartAttack,Outcome,State,PatientID
0,31,Male,Low,162,False,False,True,False,False,True,False,Unhealthy,Low,Middle,False,False,United States,1
1,69,Male,Normal,148,True,True,True,False,False,False,False,Unhealthy,High,Middle,False,False,United States,2
2,34,Female,Low,161,True,False,False,True,True,False,True,Healthy,Medium,High,True,True,United States,3
3,53,Male,High,134,True,False,True,True,False,True,False,Healthy,Medium,Middle,True,False,United States,4
4,57,Female,Normal,140,True,False,True,False,False,True,False,Moderate,Low,Middle,True,True,United States,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372969,49,Female,High,131,False,False,True,True,True,False,False,Unhealthy,Medium,High,True,False,United States,372970
372970,77,Male,Low,159,True,False,True,False,True,True,False,Unhealthy,High,High,False,True,United States,372971
372971,53,Female,High,94,True,True,True,True,False,False,False,Moderate,High,High,True,True,United States,372972
372972,78,Female,High,93,False,True,True,False,True,True,False,Healthy,High,High,False,False,United States,372973


In [58]:
# MIRI DID THIS

india['Gender'] = india['Gender'].astype('category')
india['Diabetes'] = india['Diabetes'].map({1:True, 0:False}).astype('bool')
india['Hypertension'] = india['Hypertension'].map({1:True, 0:False}).astype('bool')
india['Obesity'] = india['Obesity'].map({1:True, 0:False}).astype('bool')
india['Smoker'] = india['Smoker'].map({1:True, 0:False}).astype('bool')
india['AlcoholConsumption'] = india['AlcoholConsumption'].map({1:True, 0:False}).astype('bool')
india['PhysicalActivity'] = india['PhysicalActivity'].map({1:True, 0:False}).astype('bool')
# HealthyDiet here --> to bool
# Cholesterol --> to categoric 
india['FamilyHistory'] = india['FamilyHistory'].map({1:True, 0:False}).astype('bool')
# StressLevel here --> to categoric
india['PreviousHeartAttack'] = india['PreviousHeartAttack'].map({1:True, 0:False}).astype('bool')
# IncomeLevel here --> to categoric
india['Outcome'] = india['Outcome'].map({1:True, 0:False}).astype('bool')
india['State'] = india['State'].astype('category')

india

,PatientID,Age,Gender,Diabetes,Hypertension,Obesity,Smoker,AlcoholConsumption,PhysicalActivity,HealthyDiet,Cholesterol,BloodPressure,FamilyHistory,StressLevel,PreviousHeartAttack,IncomeLevel,Outcome,State
0,1,42,Female,False,False,True,True,False,False,9,248,93,False,4,False,611025,False,India
1,2,26,Male,False,False,False,False,True,True,4,272,134,False,7,False,174527,False,India
2,3,78,Male,False,False,True,False,False,True,6,268,104,True,10,False,1760112,False,India
3,4,58,Male,True,False,True,False,False,True,9,224,91,False,1,True,1398213,False,India
4,5,22,Male,False,False,False,False,False,True,5,277,141,False,9,False,97987,True,India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,20,Female,False,False,False,True,False,True,6,160,133,True,10,False,1175023,False,India
9996,9997,34,Female,False,False,False,False,False,False,5,257,124,True,1,False,999441,True,India
9997,9998,76,Male,False,False,False,False,False,False,2,205,151,False,1,True,159536,False,India
9998,9999,52,Male,False,False,False,False,False,True,5,155,127,False,7,False,153207,False,India


In [63]:
# MIRI DID THIS

indonesia['Gender'] = indonesia['Gender'].astype('category')
indonesia['IncomeLevel'] = indonesia['IncomeLevel'].astype('category')   # CATEGORIA LASCIATE COME TROVATE
indonesia['Hypertension'] = indonesia['Hypertension'].map({1:True, 0:False}).astype('bool')
indonesia['Diabetes'] = indonesia['Diabetes'].map({1:True, 0:False}).astype('bool')
# Cholesterol --> to categoric 
indonesia['Obesity'] = indonesia['Obesity'].map({1:True, 0:False}).astype('bool')
indonesia['FamilyHistory'] = indonesia['FamilyHistory'].map({1:True, 0:False}).astype('bool')
# Smoker --> to bool
# AlcoholConsumption --> to bool 
# indonesia['PhysicalActivity'] = indonesia['PhysicalActivity'].map({1:True, 0:False}).astype('bool')
indonesia['Diabetes'] = indonesia['Diabetes'].map({'Healthy':True, 'Unhealthy':False}).astype('bool')
# StressLevel here --> to categoric
indonesia['PreviousHeartAttack'] = indonesia['PreviousHeartAttack'].map({1:True, 0:False}).astype('bool')
indonesia['Outcome'] = indonesia['Outcome'].map({1:True, 0:False}).astype('bool')
indonesia['State'] = indonesia['State'].astype('category')

indonesia

,Age,Gender,IncomeLevel,Hypertension,Diabetes,Cholesterol,Obesity,FamilyHistory,Smoker,AlcoholConsumption,PhysicalActivity,HealthyDiet,StressLevel,BloodPressure,PreviousHeartAttack,Outcome,State,PatientID
0,60,Male,Middle,False,True,211,False,False,Never,NaN,High,Unhealthy,Moderate,113,False,False,Indonesia,1
1,53,Female,Low,False,True,208,False,True,Past,NaN,Moderate,Healthy,High,132,True,False,Indonesia,2
2,62,Female,Low,False,True,231,True,True,Past,Moderate,Moderate,Healthy,Low,116,False,True,Indonesia,3
3,73,Male,Low,True,True,202,False,True,Never,Moderate,Moderate,Unhealthy,High,136,False,False,Indonesia,4
4,52,Male,Middle,True,True,232,False,False,Current,Moderate,Moderate,Unhealthy,Moderate,127,True,True,Indonesia,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158350,60,Female,Middle,False,True,199,False,False,Never,NaN,High,Healthy,High,127,False,False,Indonesia,158351
158351,64,Female,Low,False,True,179,False,False,Never,NaN,Moderate,Healthy,High,132,False,True,Indonesia,158352
158352,30,Male,Middle,False,True,220,False,False,Past,Moderate,Low,Healthy,Moderate,133,True,True,Indonesia,158353
158353,74,Male,Middle,False,True,180,False,True,Past,NaN,Low,Unhealthy,High,134,True,True,Indonesia,158354


In [64]:
indonesia.dtypes

Age                       int64
Gender                 category
IncomeLevel            category
Hypertension               bool
Diabetes                   bool
Cholesterol               int64
Obesity                    bool
FamilyHistory              bool
Smoker                   object
AlcoholConsumption       object
PhysicalActivity         object
HealthyDiet              object
StressLevel              object
BloodPressure             int64
PreviousHeartAttack        bool
Outcome                    bool
State                  category
PatientID                 int64
dtype: object

In [17]:
# PatientID (integer), Age (integer), Gender (categoric), Smoker (bool), Hypertension (bool), Diabetes (bool), Obesity (BMI conversion from number to bool),
# Cholesterol (from number to categoric), PhisicalActivity (from * to bool), HealthyDiet (from * to bool),
# StressLevel (from * to categoric), AlcoholConsumption (from * to bool), FamilyHistory (bool), IncomeLevel (from * to categoric)
# BloodPressure (integer), PreviousHearthAttack (bool), Outcome (bool).

In [18]:
#china.set_index(['Stato','Patient_ID'], inplace=True)
#unitedStates.set_index(['Stato','Patient_ID'], inplace=True)
#india.set_index(['Stato','Patient_ID'], inplace=True)
#indonesia.set_index(['Stato','Patient_ID'], inplace=True)
#dataset = pd.concat([china, india, unitedStates, indonesia], verify_integrity=True)